In [12]:
!pip install razdel
# This is rule-based system for Russian sentence and word tokenization.

In [13]:
import pandas as pd
from collections import defaultdict, Counter
import json
from razdel import tokenize
from copy import deepcopy

train_data = pd.read_json('/kaggle/input/assignment3/train.jsonl', lines=True)
dev = pd.read_json('/kaggle/input/assignment3/dev.jsonl', lines=True)
test = pd.read_json('/kaggle/input/assignment3/test.jsonl', lines=True)

# columns renaming
dev.rename(columns={"senences": "sentences"}, inplace=True)
test.rename(columns={"senences": "sentences"}, inplace=True)

In [14]:
from collections import Counter

def train(train_data):
    my_dict = dict()

    for _, row in train_data.iterrows():
        for start, end, label in row['ners']:
            token = row['sentences'][start:end+1]

            if token not in my_dict:
                my_dict[token] = Counter()

            my_dict[token][label] = my_dict[token].get(label, 0) + 1
    return my_dict

In [15]:
train_dict = train(train_data)

In [18]:
def predict(row):
    """
    This function makes prediction
    """
    # Tokenize sentence
    tokens = list(tokenize(row['sentences']))
    ners = []
    # Each "token" contains start index, end index and token
    for token in tokens:
        start, end, text = token
        end -= 1

        # If we found token in dict then we should find Label of this token
        if text in train_dict:
            ners.append([start, end, train_dict[text].most_common(1)[0][0]])
    return ners

In [19]:
test.apply(predict, axis=1)
test.drop(['sentences'], axis=1, inplace=True)

In [20]:
# Save the results
output_path = 'test.jsonl'
with open(output_path, "w") as f:
    f.write(test.to_json(orient='records', lines=True, force_ascii=False))
!zip test test.jsonl

  adding: test.jsonl (deflated 82%)
